### Evaluating the model on other rows

In [2]:
import re
import pickle

import pandas as pd

In [3]:
file = open('df_extracted_basic', 'rb')
cv_df = pickle.load(file)

In [7]:
cv_df['CLNREVSTAT'].value_counts()

criteria_provided,_single_submitter                     231953
criteria_provided,_multiple_submitters,_no_conflicts     48754
no_assertion_criteria_provided                           36329
criteria_provided,_conflicting_interpretations           16434
no_assertion_provided                                    10274
reviewed_by_expert_panel                                  9062
no_interpretation_for_the_single_variant                   512
practice_guideline                                          23
Name: CLNREVSTAT, dtype: int64

In [7]:
# narrow rows down to just unambiguous clinical diagnoses
unamb_stat = ['criteria_provided,_multiple_submitters,_no_conflicts','reviewed_by_expert_panel','practice_guideline ']
cv_df = cv_df.loc[cv_df['CLNREVSTAT'].isin(unamb_stat)]

In [8]:
# drop rows with just drug_response as a CLNSIG
cv_df = cv_df.drop(cv_df[cv_df['CLNSIG'] == 'drug_response'].index)

#convert ['CLNSIG'] into just three catagories
cv_df.loc[cv_df['CLNSIG'].str.startswith("Benign") | cv_df['CLNSIG'].str.startswith("Likely_benign"), ['CLNSIG']] = 'Benign_cat'
cv_df.loc[cv_df['CLNSIG'].str.startswith("Uncertain_significance"), ['CLNSIG']] = 'Uncertain_significance_cat'
cv_df.loc[cv_df['CLNSIG'].str.startswith("Pathogenic") | cv_df['CLNSIG'].str.startswith("Likely_pathogenic"), ['CLNSIG']] = 'Pathogenic_cat'

In [9]:
cv_df['CLNSIG'].value_counts()

Benign_cat                    28086
Uncertain_significance_cat    17223
Pathogenic_cat                12354
Name: CLNSIG, dtype: int64

#### Drop columns absolutely not needed, and save cleaned up file as pickled file

In [10]:
#drop columns not needed
cv_df = cv_df.drop(columns=['ALLELEID', 'CLNDISDB',
       'CLNHGVS', 'CLNREVSTAT', 'CLNVCSO', 'CLNVI',
       'GENEINFO', 'RS', 'CLNSIGCONF',
       'CLNDISDBINCL', 'CLNSIGINCL', 'SSR', 'DBVARID','Allele', 
       'IMPACT', 'SYMBOL', 'Gene','Feature_type','Feature','HGVSc','HGVSp','Existing_variation','DISTANCE',
       'STRAND','FLAGS','SYMBOL_SOURCE','HGNC_ID','TSL','APPRIS','REFSEQ_MATCH',
       'GIVEN_REF','USED_REF','MOTIF_NAME','MOTIF_POS','HIGH_INF_POS','MOTIF_SCORE_CHANGE','MPC'])

In [11]:
file = open('cv_df_extracted', 'wb')
pickle.dump(cv_df, file)
file.close()